# Combining original and flipped segmentations to get purely right-ventricle segmentations

In [21]:
from pathlib import Path
import sys
import math

import cv2
import numpy as np
from tqdm import tqdm

import echonet

# print(cv2.getBuildInformation())

In [22]:
LEFT_SEGMENT_DIR = Path("output/segmentation/all-patients")
RIGHT_SEGMENT_DIR = Path("output/segmentation/flipped")
ECHONET_VIDEO_DIR = Path("/home/lex/data/echonet-data/Videos")
OUTPUT_DIR = Path("output/right-only-segmentation")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

RED = np.array([255, 0, 0])
GREEN = np.array([0, 255, 0])
BLUE = np.array([0, 0, 255])
ORANGE = np.array([255, 165, 0])
LIGHT_GREY = np.array([211, 211, 211])
MAGENTA = np.array([255, 0, 255])
YELLOW = np.array([255, 255, 0])

DO_SAVE_RIGHT_ONLY_SEGMENTATIONS = False

In [23]:
def get_heights(mask: np.ndarray):
    """Returns array of heights of mask for each frame!"""
    frame_indices, row_indices = np.where(mask.any(axis=1)==True)

    heights = []
    for frame_index in range(mask.shape[0]):
        this_frame = frame_indices == frame_index

        if len(row_indices[this_frame]) == 0:
            # print(f"Frame #{frame_index} appears to have no segmentations? Treating this as zero height...")
            heights.append(0)
            continue

        min_row, max_row = (min(row_indices[this_frame]), max(row_indices[this_frame]))
        height = max_row - min_row
        heights.append(height)

    heights = np.array(heights)
    return heights

In [24]:
if DO_SAVE_RIGHT_ONLY_SEGMENTATIONS:
    left_segment_mask_fps = [p for p in (LEFT_SEGMENT_DIR / "segmentation_masks").iterdir()]
    for left_segment_mask_fp in tqdm(left_segment_mask_fps):
        # Get associated segment masks for this particular video
        # video_mask_name = video_fp.with_suffix(".npy").name # Turns "path/to/video.avi" into "video.npy"
        video_stem = Path(left_segment_mask_fp.stem) # turns "path/to/0x12345.npy" into just "0x12345"
        right_segment_mask_fp = RIGHT_SEGMENT_DIR / "segmentation_masks" / video_stem.with_suffix(".npy")
        echonet_video_fp = ECHONET_VIDEO_DIR / video_stem.with_suffix(".avi")
        # print(f"Processing {echonet_video_fp}...")

        left_segment_mask = np.load(left_segment_mask_fp)
        right_segment_mask = np.load(right_segment_mask_fp)
        echonet_video = echonet.utils.loadvideo(str(echonet_video_fp)) # (colours, frames, height, width)

        # Since right mask comes from flipped version of video, we need to flip it back to normal (i.e. left-to-right)
        right_segment_mask = np.flip(right_segment_mask, axis=-1)

        # Keep track of the mistaken left ventricle segmentation by the "right" segmentation mask
        mistaken_left_mask = left_segment_mask & right_segment_mask
        # Subtract those mistaken left ventricle parts from the right segmentation
        right_only_mask = right_segment_mask ^ (mistaken_left_mask)

        # Get heights of left and right ventricles, check if right ventricle appears unreasonably large (i.e. probably including atrium accidentally!)
        left_heights = get_heights(left_segment_mask)
        right_heights = get_heights(right_only_mask)
        frames_with_too_large_RV_mask = right_heights > 0.8 * left_heights

        # Put colour channels at end to make it easier to assign pixel colours
        echonet_video = echonet_video.transpose((1, 2, 3, 0)) # I.e. now (frames, height, width, colours)
        echonet_video[left_segment_mask] = RED
        echonet_video[right_only_mask] = BLUE
        echonet_video[mistaken_left_mask] = GREEN
            
        border_thickness = 2
        echonet_video[frames_with_too_large_RV_mask, 0:border_thickness, :] = ORANGE
        echonet_video[frames_with_too_large_RV_mask, -border_thickness:, :] = ORANGE
        echonet_video[frames_with_too_large_RV_mask, :, 0:border_thickness] = ORANGE
        echonet_video[frames_with_too_large_RV_mask, :, -border_thickness:] = ORANGE

        # Transpose video back to original shape now and save
        echonet_video = echonet_video.transpose((3, 0, 1, 2)) # now (colours, frames, height, width) as before
        output_fp = OUTPUT_DIR / video_stem.with_suffix(".avi")

        echonet.utils.savevideo(str(output_fp), echonet_video, 30)

# Filtering out garbage data
Here, we'll be trying to discard any videos that are not A4C views.

In [25]:
def make_line(p1, p2, xs):
    x1, y1 = p1
    x2, y2 = p2
    
    gradient = (y2 - y1) / (x2 - x1)
    ys = gradient * (xs - x1) + y1
    ys = np.intp(ys)
    
    coords_list = list(zip(xs, ys))
    return coords_list

In [26]:
def get_min_area_box(image):
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume we're only interested in max area contour
    areas = [cv2.contourArea(cnt) for cnt in contours]
    max_index = np.argmax(areas)
    biggest_contour = contours[max_index]

    min_area_rect = cv2.minAreaRect(biggest_contour)
    box = np.intp(cv2.boxPoints(min_area_rect))
    
    return box

In [27]:
def distance_between_two_parallel_lines(points1, points2):
    (x1, y1), (x2, y2) = points1
    (x3, y3), (x4, y4) = points2

    # Double check that gradients of these two lines are the same
    m = (y2 - y1)/(x2 - x1)
    # assert m == (y4 - y3)/(x4 - x3)

    # y - y1 = m(x - x1)
    # y = mx - m x1 + y1
    # y = mx + (y1 - m x1) == mx + c1
    # d = |c2 - c1| / sqrt(m^2 + 1)
    c1 = y1 - m * x1
    c2 = y3 - m * x3
    d = abs(c2 - c1) / math.sqrt(m**2 + 1)
    return d

    # # Starting with y - y1= m(x - x1), we end up with
    # # -x + (1/m) y + (x1 - y1/m) = 0, which we identify as
    # # ax + by + c = 0
    # c1 = x1 - y1 / m # c for first line
    # c2 = x3 - y3 / m # c fo second line
    # a = -1
    # b = 1 / m

    # # Distance between two parallel lines then given by the following
    # d = abs(c1 - c2) / math.sqrt(a**2 + b**2)
    # # d = abs(x1 - x2 + y2/m - y1/m) / math.sqrt(1/m**2 + 1)
    # return d

In [28]:
VIDEONAME = "0X1A58C9DFE12C7953"

LEFT_SEGMENTATION_MASK_FP = Path(f"/home/lex/Development/ultrasound/output/segmentation/all-patients/segmentation_masks/{VIDEONAME}.npy")
FLIPPED_SEGMENTATION_MASK_FP = Path(f"/home/lex/Development/ultrasound/output/segmentation/flipped/segmentation_masks/{VIDEONAME}.npy")
ECHONET_VIDEO_FP = Path(f"/home/lex/data/echonet-data/Videos/{VIDEONAME}.avi")

left_segmentation_mask = np.load(LEFT_SEGMENTATION_MASK_FP)
flipped_segmentation_mask = np.load(FLIPPED_SEGMENTATION_MASK_FP)
flipped_segmentation_mask = np.flip(flipped_segmentation_mask, -1)
intersection_mask = left_segmentation_mask & flipped_segmentation_mask
# Subtract those mistaken left ventricle parts from the right segmentation
right_segmentation_mask = flipped_segmentation_mask ^ intersection_mask
# right_segmentation_mask = right_segmentation_mask.astype(np.uint8) * 255

echonet_video = echonet.utils.loadvideo(str(ECHONET_VIDEO_FP))
echonet_video = echonet_video.transpose((1, 2, 3, 0))

num_frames, height, width, num_channels = echonet_video.shape

In [29]:
# Calculate RV areas
right_segmentation_mask_image = right_segmentation_mask.astype(np.uint8) * 255
right_contours_list = [cv2.findContours(frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) for frame in right_segmentation_mask_image]
right_contours_list = [x[0] for x in right_contours_list]

right_max_areas = []
for i, right_contours in enumerate(right_contours_list):
    # print(f"i = {i}")
    right_areas = [cv2.contourArea(contour) for contour in right_contours]
    max_area = max(right_areas)
    right_max_areas.append(max_area)

In [30]:
# Calculate estimated septum widths using distance between left and right segmentations' tightest boxes
septum_widths = np.zeros(num_frames)
for i, (left_mask, right_mask) in enumerate(zip(left_segmentation_mask, right_segmentation_mask)):
    left_segmentation = left_mask.astype(np.uint8) * 255
    right_segmentation = right_mask.astype(np.uint8) * 255

    LV_box = get_min_area_box(left_segmentation)
    RV_box = get_min_area_box(right_segmentation)
    top_left_LV, top_right_LV, bottom_right_LV, bottom_left_LV = LV_box
    top_left_RV, top_right_RV, bottom_right_RV, bottom_left_RV = RV_box

    dist = distance_between_two_parallel_lines([top_right_RV, bottom_right_RV], [top_left_LV, bottom_left_LV])
    septum_widths[i] = dist

# Will get NAN distances when the two bounding boxes touch each other (leads to divide by zero), so set those to distance zero
septum_widths[np.isnan(septum_widths)] = 0

print(f"Mean septum width: {np.mean(septum_widths)} +/- {np.std(septum_widths)}")


Mean septum width: 5.811004901334838 +/- 2.477535625421556


/tmp/ipykernel_865457/4048570368.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  m = (y2 - y1)/(x2 - x1)
/tmp/ipykernel_865457/4048570368.py:15: RuntimeWarning: invalid value encountered in scalar subtract
  d = abs(c2 - c1) / math.sqrt(m**2 + 1)


In [31]:
WINDOW = "Mask"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)
cv2.namedWindow("Erosion", cv2.WINDOW_NORMAL)
cv2.namedWindow("Dilation", cv2.WINDOW_NORMAL)
cv2.namedWindow("Denoised", cv2.WINDOW_NORMAL)

i = 0
is_playing = True

while True:
    if i >= num_frames:
        i = 0
        print("Video looped!")
    elif i < 0:
        i = num_frames - 1

    frame = echonet_video[i].copy()
    ret, denoised = cv2.threshold(frame,20,255,cv2.THRESH_TOZERO)
    denoised = cv2.fastNlMeansDenoisingColored(denoised,None,10,10,7,21)
    # denoised = cv2.threshold(denoised,40,255,cv2.THRESH_TOZERO)
    cv2.imshow("Denoised", denoised)

    frame_left_segmentation_mask = left_segmentation_mask[i]
    frame_right_segmentation_mask = right_segmentation_mask[i].copy()
    frame_intersection_segmentation_mask = intersection_mask[i]

    # frame[frame_left_segmentation_mask] = RED
    # frame[frame_intersection_segmentation_mask] = GREEN

    # Find tightest box around LV segmentation
    frame_left_segmentation = frame_left_segmentation_mask.astype(np.uint8) * 255
    LV_box = get_min_area_box(frame_left_segmentation)
    cv2.drawContours(frame,[LV_box],0, MAGENTA.tolist())
    top_left_LV, top_right_LV, bottom_right_LV, bottom_left_LV = LV_box

    cutoff_x = np.intp(np.arange(0, frame.shape[1]))
    bottom_cutoff_points = make_line(bottom_left_LV, bottom_right_LV, cutoff_x)
    cv2.line(frame, bottom_cutoff_points[0], bottom_cutoff_points[-1], MAGENTA.tolist())
    
    top_cutoff_points = make_line(top_left_LV, top_right_LV, cutoff_x)
    cv2.line(frame, top_cutoff_points[0], top_cutoff_points[-1], MAGENTA.tolist())
    
    right_cutoff_points_untrimmed = make_line(bottom_left_LV, top_left_LV, cutoff_x)
    right_cutoff_points = []
    for x, y in right_cutoff_points_untrimmed:
        if y >=0 and y < frame.shape[1]:
            right_cutoff_points.append((x, y))

    cv2.line(frame, right_cutoff_points[0], right_cutoff_points[-1], MAGENTA.tolist())

    for x, y in bottom_cutoff_points:
        frame_right_segmentation_mask[y:, x] = False
    for x, y in right_cutoff_points:
        frame_right_segmentation_mask[:y, x] = False

    frame[frame_right_segmentation_mask] = BLUE

    # Draw box around RV segmentation, and determine shortest distance to LV box
    frame_right_segmentation = frame_right_segmentation_mask.astype(np.uint8) * 255
    RV_box = get_min_area_box(frame_right_segmentation)
    cv2.drawContours(frame,[RV_box],0, YELLOW.tolist())
    septum_width = septum_widths[i]
    print(f"Septum width: {septum_width:.2f}")
    
    # kernel = np.ones((3, 3))
    # kernel = np.array([
    #     [1, 0, 0],
    #     [0, 1, 0],
    #     [0, 0, 1]
    # ], dtype=np.uint8)
    kernel = np.array([
        [1, 1],
        [1, 1]
    ], dtype=np.uint8)
    # kernel = np.array([
    #     [0, 0, 0, 0, 1, 1],
    #     [0, 0, 0, 1, 1, 1],
    #     [0, 0, 1, 1, 1, 1],
    #     [0, 1, 1, 1, 1, 1],
    #     [1, 1, 1, 1, 1, 1],
    #     [1, 1, 1, 1, 1, 1],
    # ], dtype=np.uint8)
    kernel = np.array([
        [0, 0, 0, 1],
        [0, 0, 1, 1],
        [0, 1, 1, 1],
        [1, 1, 1, 1],
    ], dtype=np.uint8)
    frame_right_segmentation = cv2.erode(frame_right_segmentation, kernel, cv2.BORDER_REFLECT, iterations=3)
    cv2.imshow("Erosion", frame_right_segmentation)

    # frame[frame_right_segmentation > 0] = ORANGE

    # Find contour with max area|
    contours, hierarchy = cv2.findContours(frame_right_segmentation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    max_index = np.argmax(areas)
    biggest_contour = contours[max_index]

    biggest_right_segmentation = np.zeros(frame_right_segmentation.shape)
    biggest_right_segmentation = cv2.drawContours(biggest_right_segmentation, [biggest_contour], -1, 255, -1)

    dilation_kernel = np.ones((8, 8))
    # dilation_kernel = np.array([
    #     [0, 0, 0, 0, 1, 1],
    #     [0, 0, 0, 1, 1, 1],
    #     [0, 0, 1, 1, 1, 0],
    #     [0, 1, 1, 1, 0, 0],
    #     [1, 1, 1, 0, 0, 0],
    #     [1, 1, 0, 0, 0, 0],
    # ], dtype=np.uint8)
    # dilation_kernel = np.array([
    #     [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    #     [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    #     [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    #     [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    #     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    #     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    #     [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    #     [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    #     [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
    #     [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    # ], dtype=np.uint8)
    dilation_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(9,9))
    dilation_kernel += dilation_kernel.transpose()
    frame_right_segmentation_dilated = cv2.dilate(biggest_right_segmentation, dilation_kernel, iterations=1, borderType=cv2.BORDER_REFLECT)
    cv2.imshow("Dilation", frame_right_segmentation_dilated)

    frame_right_segmentation_dilated_mask = frame_right_segmentation_dilated > 0
    overlap_mask = frame_right_segmentation_mask & frame_right_segmentation_dilated_mask
    # frame[overlap_mask] = ORANGE
    # difference_mask = frame_right_segmentation_dilated_mask & (~frame_right_segmentation_mask)
    # frame[difference_mask] = MAGENTA

    this_area = right_max_areas[i]
    if i > 0: 
        prev_area = right_max_areas[i - 1]
        growth = this_area/ prev_area
        print(f"RV Area growth: {growth:.2f}")
        if growth > 1.2:
            frame[:10, :] = ORANGE

    top_border = np.zeros((height // 8, width, 3), dtype=frame.dtype)
    top_border[:, :] = np.expand_dims(LIGHT_GREY, (0, 1))
    cv2.putText(top_border, f"Frame {i+1}/{num_frames}", org=(5,10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.25, color=(255, 0, 0))
    frame = np.concatenate([top_border, frame], axis=0)
    frame = np.flip(frame, -1)
    cv2.imshow(WINDOW, frame)

    keypress = cv2.waitKey(50) & 0xFF
    if keypress == ord('q'):
        break
    elif keypress == ord(' '):
        is_playing = not is_playing
    elif keypress == ord('a'):
        i -= 1
    elif keypress == ord('d'):
        i += 1
    else:
        if is_playing:
            i += 1

cv2.destroyAllWindows()

Septum width: 6.50
Septum width: 8.49
RV Area growth: 0.93
Septum width: 9.51
RV Area growth: 1.01
Septum width: 10.04
RV Area growth: 1.01
Septum width: 10.42
RV Area growth: 0.98
Septum width: 9.00
RV Area growth: 0.99
Septum width: 7.87
RV Area growth: 0.97
Septum width: 5.16
RV Area growth: 1.07
Septum width: 0.00
RV Area growth: 1.04
Septum width: 0.00
RV Area growth: 0.92
Septum width: 0.00
RV Area growth: 0.98
Septum width: 3.61
RV Area growth: 0.95
Septum width: 3.75
RV Area growth: 0.95
Septum width: 2.30
RV Area growth: 0.98
Septum width: 2.31
RV Area growth: 0.90
Septum width: 1.00
RV Area growth: 0.94
Septum width: 4.03
RV Area growth: 0.98
Septum width: 0.00
RV Area growth: 1.07
Septum width: 0.00
RV Area growth: 0.95
Septum width: 2.06
RV Area growth: 0.99
Septum width: 3.12
RV Area growth: 1.06
